In [1]:
import os, re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import platform, time, urllib, random
from selenium import webdriver as wd

In [ ]:
# 환경 설정.
# Chrome 브라우저 띠우기.
chromedriver ='c:/chromedriver.exe'

# Mac 환경.
if platform.system() == 'Darwin': 
    # 맥 PC는 드라이버 있는 위치에서 아래 명령 터미널에서 수행 후 진행.
    # $ xattr -d com.apple.quarantine chromedriver
    driver = wd.Chrome(chromedriver)
    driver.implicitly_wait(3)

# Window 환경.  
else:   
    driver = wd.Chrome(chromedriver)
    driver.implicitly_wait(3)

In [ ]:
# 사이트 로그인 함수.
def Login() :
    # EveryTime 접속을 위한 자동 로그인 설정.
    URL = 'https://everytime.kr/login'
    driver.get(URL)
    
    user_id = input('Input your ID : ')
    user_pw = input('Input your Password : ')
    driver.find_element_by_name('userid').send_keys(user_id)
    driver.find_element_by_name('password').send_keys(user_pw)

    # Element 접근.
    driver.find_element_by_xpath('//*[@class="submit"]/input').click()
    
    # Uniform 분포로 부터 랜덤 시간값 설정.
    value = random.uniform(3, 7) 
    time.sleep(value)

In [ ]:
# 테스트 시에는 입력값 미리 입력시킴으로써 자동 로그인화하고 진행.
# 로그인 하기.
Login()

In [ ]:
# 시간표 URL로 이동.
driver.find_element_by_xpath('//*[@id="menu"]/li[2]/a').click()
value = random.uniform(2, 5) 
time.sleep(value)

# 년도 및 학기 선택 여부 가져오기.
year, sem = input('수집하고자 하는 년도와 학기를 입력해주세요. ex) 2021, 1   : 사용자 입력값 : ').split(',')  

url = "https://everytime.kr/timetable/" + year + '/' + sem
value = random.uniform(3, 5) 
time.sleep(value)
driver.get(url)

# 수업 목록에서 검색 버튼 클릭.
driver.find_element_by_xpath('//*[@id="container"]/ul/li[1]').click()
value = random.uniform(2, 5) 
time.sleep(value)

# 처음 뜨는 팝업창 닫기.
# 처음 들어가는 것이 아니라면 이 부분은 작동시키지 않음.
# river.find_element_by_xpath('//*[@id="sheet"]/ul/li[3]').click()
# value = random.uniform(2, 5) 
# time.sleep(value)


# 전공/영역 클릭.
driver.find_element_by_xpath('//*[@id="subjects"]/div[1]/a[3]').click()
value = random.uniform(2, 5) 
time.sleep(value)

# 2018년도 2학기 부터 전공 버튼 클릭이 존재.
# 2018년 1학기 이전은 다른 방식으로 수집하도록 설정할 필요가 있음.
# 전공 클릭.
driver.find_element_by_xpath('//*[@id="subjectCategoryFilter"]/div/ul/li[4]').click()
value = random.uniform(2, 5) 
time.sleep(value)

# 여기서 부터 반복문 수행할 필요가 있음.
# 전공 재스크롤 => 전공 선택(이과정이 2번째부터는 필요x) => 선택한 전공 끝까지 스크롤 => 선택 전공 수집 및 저장 => 전공 영역 선택.
process1 = True
process2 = True
while process1 :
    
    # 데이터 수집 여부 질문.
    choice = input('데이터를 수집을 진행하시겠습니까? :').lower()
    if choice == '네' or choice == 'yes' :
        pass
    elif choice == '아니오' or choice == 'no' :
        process = False 
        break
    else :
        print('정확하게 입력하세요.')
        continue
    
    # 학과 선택.
    # 학과 선택을 하면 이에 대응하는 학과 번호를 찾아 사이트를 이동.
    sub_num_list = {
                     '학과'    : [],
                     '학과 ID' : []
                        }
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    ul = soup.select('#subjectCategoryFilter > div > ul > ul.unfolded')
    for li in ul :
        a1 = li.select('li')
        for idx, a2 in enumerate(a1) :
            a3 = a2.text.strip()
            sub_num_list['학과'].append(a3)
            sub_num_list['학과 ID'].append(idx + 1)
    data = pd.DataFrame(sub_num_list)
    
    # while process2 :
    sub_name = input('학과를 선택하십시오. : ').lower()
    info = data['학과'].values

    if any(sub_name == i for i in info) :
        print('수집을 시작합니다.')
        break
    else : 
        print('해당되는 과목명이 존재하지 않습니다.')
        continue    

    select_data = data[data['학과'] == sub_name]
    sub_num = select_data['학과 ID'].values[0]

    # 전공 목록 스크롤 내리기.
    element_list = []
    for i in range(1, 148) :
        element = driver.find_elements_by_css_selector(f'#subjectCategoryFilter > div > ul > ul.unfolded > li:nth-child({i})')
        element_list.append(element)

    for i in range(1, 150, 30) :
        current_cnt = element_list[i][0]
        driver.execute_script('arguments[0].scrollIntoView(true);', current_cnt)
        value = random.uniform(1, 2) 
        time.sleep(value)

    # 끝나는 시점 설정.
    result = element_list[-1][0]

    if result == current_cnt :
        break
    else :
        result = current_cnt

    driver.find_element_by_xpath(f'//*[@id="subjectCategoryFilter"]/div/ul/ul[3]/li[{sub_num}]').click()
    value = random.uniform(2, 5) 
    time.sleep(value)

    # 검색하여 들어간 해당과목 스크롤 맨아래까지 내리기.
    cnt = 0
    while True :
        element = driver.find_elements_by_css_selector("#subjects > div.list > table > tbody > tr")
        end     = element[-1]
        driver.execute_script('arguments[0].scrollIntoView(true);', end)
        value   = random.uniform(1, 2) 
        time.sleep(value)

        # 끝나는 시점.
        current_count = len(element)
        if cnt == current_count :
            break
        cnt = current_count

    # 과목에 관한 자료 수집.
    html   = driver.page_source
    soup   = BeautifulSoup(html, 'html.parser')
    tr_tag = soup.select('#subjects > div.list > table > tbody > tr')

    subjet_dict = {
                    '학과'       : [],
                    '학년'       : [],
                    '구분'       : [],
                    '과목코드'   : [],
                    '강의명'     : [],
                    '학점'       : [],
                    '총시간'     : [],
                    '교수명'     : [],
                    '강의실/시간': [],
                    '강의평'     : [],
                    '담은인원'   : [],
                    '정원'       : [],
                    '수강방법'   : [],
                    'URL_ID'     : []
                    }
    num = []
    for idx, tr in enumerate(tr_tag) :

        td = tr.select('#subjects > div.list > table > tbody > tr > td')
        a1 = tr.select('td:nth-child(9) > a')
        a2 = a1[0].attrs['title']
        a3 = a1[0].attrs['href'].split('/')[-1]
        num.append(idx)

        subjet_dict['학년'].append(td[0].text.strip()) 
        subjet_dict['구분'].append(td[1].text.strip()) 
        subjet_dict['과목코드'].append(td[2].text.strip())
        subjet_dict['강의명'].append(td[3].text.strip())
        subjet_dict['학점'].append(td[4].text.strip())
        subjet_dict['총시간'].append(td[5].text.strip()) 
        subjet_dict['교수명'].append(td[6].text.strip())
        subjet_dict['강의실/시간'].append(td[7].text.strip())
        subjet_dict['강의평'].append(a2)
        subjet_dict['담은인원'].append(td[9].text.strip())
        subjet_dict['정원'].append(td[10].text.strip())
        subjet_dict['수강방법'].append(td[11].text.strip()) 
        subjet_dict['URL_ID'].append(a3)

    for s in soup.select('#subjects > div.filter > a.item.active > span.value') :
        subjet_dict['학과'].append([s.text.strip()] * (num[-1] + 1) )

    subjet_dict['학과'] = subjet_dict['학과'][0]

    # 수집한 자료 저장.
    df = pd.DataFrame(subjet_dict) 

    if os.path.exists('data.csv') == False :
        df.to_csv('data.csv', encoding='utf-8-sig', index=False)  
    else :
        # 수집한 자료 추가하기.
        df.to_csv('data.csv', encoding='utf-8-sig', index=False, header=None, mode='a')

    name = subjet_dict['학과'][0]
    print(f'{name} 데이터 저장완료')
    print('=' * 130)

    # 전공 재클릭.
    driver.find_element_by_xpath('//*[@id="subjects"]/div[1]/a[3]').click()
    value = random.uniform(2, 5) 
    time.sleep(value)

print('데이터 수집을 종료하였습니다.')

In [ ]:
# 수집한 데이터 읽어 오기.
df = pd.read_csv('data.csv')

# df2는 자연어 처리를 위한 댓글이 달린 데이터만 가져옴.
df2 = df[df['강의평'] != 0]
df2.shape

In [ ]:
# Url_id 문자열 값으로 리스트 변환.
url_id_List = list(map(str, df2['URL_ID'].values))

# 같은 교수가 같은 과목을 2개 이상 강의 하는 경우가 다소 존재.
# 즉, 데이터 중복을 고려하여 동일한 URL_ID 값 제거.
url_id_List = set(url_id_List)
len(url_id_List)

In [ ]:
# 데이터 수집 함수.
def Information(url_id_List) :
    Review_dict = {
                    "url_id" : [],
                    "번호"   : [],
                    "강의명" : [],
                    "교수명" : [],
                    "평점"   : [],
                    "년도"   : [],
                    "학기"   : [],
                    "추천수" : [],
                    "댓글"   : []
                }

    for url_id in url_id_List :

        # 선택 강의 경로로 넘어가기.
        url = "https://everytime.kr/lecture/view/" + url_id
        value = random.uniform(3, 5) 
        time.sleep(value)
        driver.get(url)

        time.sleep(3)
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')

        lecture_name = soup.select_one("#container > div.side.head > h2").text
        professor_name = soup.select_one("#container > div.side.head > p:nth-of-type(1) > span").text
        review_list = soup.select("article")
        # print(lecture_name, professor_name)
        # print(review_list)

        # 각 강의당 리뷰.
        index = 0
        for review in review_list :
            if review.select_one('div.pay') == None : 

                rate = review.select_one('p.rate > span > span')["style"]      
                rate = float(rate[-5 : -2])                                
                # print(rate)

                semester = review.select_one('p.info > span.semester').text
                year = int(semester[0:2])
                semester = int(semester[4])
                text = review.select_one('p.text').text
                like = review.select_one("p.info > span.posvote")
                if like == None :
                    like = 0
                else :
                    like = int(like.text)

                Review_dict['url_id'].append(url_id)
                Review_dict['번호'].append(index)
                Review_dict['강의명'].append(lecture_name)
                Review_dict['교수명'].append(professor_name)
                Review_dict['평점'].append(rate)
                Review_dict['년도'].append(year)
                Review_dict['학기'].append(semester)
                Review_dict['추천수'].append(like)
                Review_dict['댓글'].append(text)

                index += 1

    # 데이터 프레임 생성.
    df = pd.DataFrame(Review_dict)

    # 댓글이 없는 데이터 자료 제거.
    # df2 = df1[df1['평가 수준'] != 0]

    if os.path.exists('review.csv') == False :
        df.to_csv('review.csv', encoding='utf-8-sig', index=False)  
    else :
        df.to_csv('review.csv', encoding='utf-8-sig', index=False, header=False, mode='a')

    print('저장 완료')

In [ ]:
# 데이터 수집 시작.
Information(url_id_List)

In [ ]:
# 종료.
driver.close()

In [ ]:
# 저장된 데이터 불러와 데이터 확인.
df3 = pd.read_csv('Data Collection/review.csv')
pd.set_option('display.max_rows', None)
df3